In [1]:
"""File to generate sample data for testing."""
from datetime import date, timedelta, datetime
import random
import math
import pandas as pd
import numpy as np
import plotly.graph_objects as go



In [2]:
from sample_data import value_sign,day,adequacy_list,choices,last_week_date, last_year_date, last_month_date,test_type, results_list, genotype_list

In [3]:
#Initialization of variables
days = 90
amplitude = 20
displacement = 50
noise = 0.4
day = date.today().day
month = date.today().month
year = date.today().year

# if month == 1:
#     year = year - 1
#     month = 12

# last_month_date = date(year, month -1, day)
# last_week_date = date.today() - timedelta(days = 7)
# last_year_date = date.today() - timedelta(days = 365)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Initial Lists
#test_type = ['liquid based', 'conventional','another type', 'fourth type', 'just another']
#adequacy_list = [{'satisfactory': 'Yes'},{'satisfactory': 'No', 'processed': 'Not processed'},{'satisfactory': 'No', 'processed': 'processed'}]
# results_list = ['negative', 'ASC-US', 'ASC-H', 'LSIL', 'MSIL', 'SCC', 'AC', 'JJJ', 'KKK', 'LLL']
# genotype_list = ['16','18']
################# second list
#test_type = ['Urine', 'Spinal Fluid','Blood', 'Pleural Fluid','Pericardial Fluid']
# adequacy_list = [{'Satisfactory': 'Yes'},{'Satisfactory': 'No', 'Processed': 'Not Processed'},{'Satisfactory': 'No', 'Processed': 'Processed'}]
# results_list = ['Negative', 'ASC-US', 'ASC-H', 'LSIL', 'MSIL', 'SCC', 'AC', 'JJJ', 'KKK', 'LLL']
# genotype_list = ['16','18']
# choices = ['True positive', 'False positive', 'False negative', 'True negative']

initial_df = pd.DataFrame()
test_df = pd.DataFrame(columns=['day', 'type', 'adequacy', 'result', 'mvp', 'cytology', 'hystology'])

In [4]:
#Data Generator
def sin_data_generate(N,A,S,R):
    """To generate sinusoidal data."""
    x = np.arange(0,N,1)
    noise = []
    for j in range(N):
        noise.append(R*random.choice(range(-3,3)))
    noise = np.array(noise)
    y = A*np.sin(30*x/(2*math.pi))+S
    y = y + noise
    return [x,y]

In [5]:
###Create initial df
for type in test_type:
    test_data = sin_data_generate(days, random.randint(5, 20), random.randint(20,50), noise)
    final = datetime.date(datetime.now())
    initial = final - timedelta(days=days-1)
    # print(initial, final)
    daterange = pd.date_range(initial, final, freq='D')

    initial_df['day'] = daterange
    initial_df[type] = test_data[1].astype(int)
    # test_df['adequacy ' + type] = random.choices(adequacy_list, k=len(test_df))
    # test_df['result ' + type] = random.choices(results_list, k=len(test_df))
    # test_df['mvp'] = random.choices(genotype_list, k=len(test_df))
    # test_df['cytology'] = random.choices(['positive', 'negative'], k=len(test_df), weights=[0.8, 0.2])
    # test_df['hystology'] = random.choices(['positive', 'negative'], k=len(test_df), weights=[0.8, 0.2])

# print(test_df.count)
#initial_df

In [6]:
print(test_df.columns)
for type in test_type:
    type_list_samples = initial_df[type].values
    for index, number in enumerate(type_list_samples):
        # print(f'number: {number}')
        # print(f'dataframe: {(initial_df[type].values)}')
        for sample in range(number):
            day = initial_df['day'].iloc[index]
            adequacy = random.choices(adequacy_list)[0]
            result = random.choices(results_list)[0]
            mvp = random.choices(genotype_list)[0]
            cytology = random.choices(value_sign, weights=[0.8, 0.2])[0]
            hystology = random.choices(value_sign, weights=[0.8, 0.2])[0]
            new_row = {'day': day,'type': type, 'adequacy': adequacy, 'result': result, 'mvp': mvp, 'cytology': cytology, 'hystology': hystology}
            # test_df = test_df.apply(lambda row: add_row(row))
            new_df = pd.DataFrame([new_row])
            test_df = pd.concat([test_df, new_df], axis=0, ignore_index=True)
            new_df.reset_index()

            # print(f'sample: {test_df["type"]}')
                # test_df = test_df.concat(new_row, ignore_index=True)
                # test_df['type'] = type

#(test_df.tail(5))
    test_df['test_quality'] = np.select([
        (test_df['cytology'] == 'positive') & (test_df['hystology'] == 'positive'),
        (test_df['cytology'] == 'positive') & (test_df['hystology'] == 'negative'),
        (test_df['cytology'] == 'negative') & (test_df['hystology'] == 'positive'),
        (test_df['cytology'] == 'negative') & (test_df['hystology'] == 'negative')
        ], choices, default='True negative')


Index(['day', 'type', 'adequacy', 'result', 'mvp', 'cytology', 'hystology'], dtype='object')


## These are the dfs to be exported

In [15]:
initial_df.to_csv('test_type_count_C.csv')
test_df.to_csv('test_dataframe_C.csv')

In [22]:
initial_df.head(5)

,day,Urine,Spinal Fluid,Blood,Pleural Fluid,Pericardial Fluid
0,2022-09-21,26,45,39,44,26
1,2022-09-22,13,29,21,29,17
2,2022-09-23,24,42,35,43,25
3,2022-09-24,36,58,57,61,37
4,2022-09-25,28,49,43,48,30


In [131]:
old_test_df=pd.read_csv("test_dataframe_Camilo.csv")

## The rest are graphing experiments

In [21]:
panda = test_df.loc[test_df['type'] == 'liquid based']
panda = panda.loc[panda['result'] == 'LLL']
frame = panda.groupby('day')
another_frame = pd.DataFrame()
another_frame['count'] = frame['result'].count()
another_frame = another_frame.reset_index()
# another_frame
graph = go.Figure()
graph.add_trace(go.Scatter(x=another_frame['day'], y= another_frame['count'], mode="markers+lines", name='LLL', marker_symbol= 1, showlegend=True))
graph


In [62]:

graph = go.Figure()
if test_type[0] == 'negative':
    visibility = True
else:
    visibility = 'legendonly'

# graph.add_trace(go.Scatter(x=test_df['day'], y=initial_df[type], mode='lines+markers', name=test_type[0], cliponaxis=True,
# line_shape= "spline", showlegend=True, line_smoothing=0, visible=visibility
#     ))
count = 0
for item in results_list:
    count += 1
    filter1 = test_df['result'] == item
    data = test_df[filter1]
    data.sort_values('day', inplace=True)
    day_group = data.groupby('day')
    # sample_df = test_df[filter1]
    count_data = day_group['result'].value_counts()
    count_data    
    
    # df = data[data['result'] == item]
    # day_grp = df.groupby(['day'])
    # df['count'] = test['result'].count()
    mock = pd.DataFrame()
    mock['day'] = data['day']
    mock.sort_values('day', inplace=True)
    mock['count'] = count_data
    # mock['count'] = mock['count'].fillna(0)
    # mock = test['type'].count()
    # graph.add_trace(go.Scatter(x=mock['day'], y= mock['count'], mode="markers+lines", name=item, marker_symbol= count, showlegend=True))
# test['type'].count()
# (type_list_samples)
# day_grp['result'].value_counts()
#mock.show()

/var/folders/_l/7xr3qhx50rl1tsc0s6dt78hr0000gn/T/ipykernel_28624/1841605538.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



TypeError: incompatible index of inserted column with frame index

In [30]:
paleta=["#FFC006","#95BFE1","#E83283","#42D7A7","#38CBFB"]

In [33]:
import plotly.graph_objects as go
screen_green="#00ff00"
orange="#FFC006"
fig = go.Figure()
fig.add_trace(go.Bar(
    y=["Adequate", 'Inadequate'],
    x=[20, 14],
    name='Processed',
    orientation='h',
    marker=dict(
        color=screen_green,
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=["Adequate","Inadequate"],
    x=[12, 18],
    name='Not Processed',
    orientation='h',
    marker=dict(
        color=orange,
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))

fig.update_layout(barmode='stack')
fig.show()